In [3]:
def loadFile(dataPath,labelPath):
    dataList = []
    labelList= []
    #打开文件：以二进制读模式、utf-8格式的编码方式打开
    frData = open(dataPath)
    frLabel = open(labelPath)
    dataRecord = frData.read()
    labelRecord = frLabel.read()
    # print(record)
    frData.close()
    frLabel.close()
    #按照行转换为一维表即包含各行作为元素的列表，分隔符有'\r', '\r\n', \n'
    dataRecordList = dataRecord.splitlines()
    labelRecordList=labelRecord.splitlines()
    #逐行遍历：行内字段按'\t'分隔符分隔，转换为列表
    for line in dataRecordList:
        if line.strip():
            a=line.split('\t')
            a.pop()
            a=list(map(float, a))
            dataList.append(a)
    for line in labelRecordList:
            if line.strip():
                a=line.split()
                # a.pop()
                a=list(map(float, a))
                if a[0]!=0:
                    labelList.append(1)
                else:
                    labelList.append(0)
            # dataList.append(list(line.split("\t")))
    #返回转换后的矩阵
    x = np.array(dataList)
    y = np.array(labelList)
    return np.absolute(x),y
    # return dataList

In [4]:
import numpy as np
dataPath ="./data.txt"
labelPath = "./label.txt"
tx,ty=loadFile(dataPath,labelPath)


In [ ]:
print(len(tx[0]))
print(len(tx),len(ty))
print(ty[:100])

In [65]:
import tensorflow as tf
from keras.engine.base_layer import Layer, InputSpec
from keras.backend import dropout
from keras.layers import GRU, Bidirectional, Dropout, TimeDistributed, Dense, Activation, GlobalAveragePooling1D, \
    Reshape, MaxPooling3D, MaxPooling1D
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
# from sklearn.model_selection import train_test_split
class NonMasking(Layer):
    """
	Non Masking Layer
    """

    def __init__(self, **kwargs):
        self.supports_masking = True
        super(NonMasking, self).__init__(**kwargs)

    def build(self, input_shape):
        input_shape = input_shape

    def compute_mask(self, input, input_mask=None):
        return None

    def call(self, x, mask=None, **kwargs):
        return x

    def compute_output_shape(self, input_shape):
        return input_shape

class KMaxPooling(Layer):
    """K-max pooling

    k-max pooling layer that extracts the k-highest activations from a sequence and calculate average
    base on Tensorflow backend.
    """
    def __init__(self, k=1, **kwargs):
        super(KMaxPooling,self).__init__(**kwargs)
        self.input_spec = InputSpec(ndim = 3)
        self.k = k

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.k, input_shape[1])

    def call(self, inputs,**kwargs):
        top_k = tf.nn.top_k(inputs, k=self.k, sorted=True, name=None)[0]
        shifted_output = tf.transpose(top_k, [0, 2, 1])
        return shifted_output

inputs = keras.Input(shape=(300,))
x = layers.Embedding(1000, 256)(inputs)
# Embed each integer in a 128-dimensional vector
# mask_1 = keras.layers.Masking(mask_value=0.0, name='mask_1')(x)
bgru_1 = Bidirectional(GRU(units=128, activation='tanh', recurrent_activation='hard_sigmoid', return_sequences=True), name='bgru_1')(x)
dropout_1 = Dropout(rate=0.4, name='dropout_1')(bgru_1)
bgru_2 = Bidirectional(GRU(units=128, activation='tanh', recurrent_activation='hard_sigmoid', return_sequences=True), name='bgru_2')(dropout_1)
dropout_2 = Dropout(rate=0.4, name='dropout_2')(bgru_2)
dense_1 = TimeDistributed(Dense(1), name='dense1')(dropout_2)
activation_1 = Activation('sigmoid', name='activation_1')(dense_1)
# unmask_1 = NonMasking()(activation_1)/
reshape_1 = Reshape((1, 300))(activation_1)
k_max_1 = MaxPooling1D(pool_size=300,data_format='channels_first')(reshape_1)
average_1 = GlobalAveragePooling1D(name='average_1')(k_max_1)
# # Add a classifier
# outputs = layers.Dense(1, activation="sigmoid")(average_1)
model = keras.Model(inputs, average_1)
model.summary()

callbacks = [
    keras.callbacks.EarlyStopping(
        # 不再提升的关注指标
        monitor='accuracy',
        # 不再提升的阈值
        min_delta=1e-3,
        # 不再提升的轮次
        patience=2,
        verbose=1
    ),
    keras.callbacks.ModelCheckpoint(
    # 模型路径
    filepath='./model/model_{epoch}.h5',
    # 是否保存最佳
    save_best_only=True,
    # 监控指标
    monitor='val_accuracy',
    # 进度条类型
    verbose=1
    )
]

model.compile(optimizer="adam",
              loss="binary_crossentropy",
              metrics=["accuracy",
                       "Precision",
                       "Recall",
                       "TruePositives",
                       "TrueNegatives",
                       "FalsePositives",
                       "FalseNegatives"])

Model: "model_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_15 (InputLayer)        [(None, 300)]             0         
_________________________________________________________________
embedding_14 (Embedding)     (None, 300, 256)          256000    
_________________________________________________________________
bgru_1 (Bidirectional)       (None, 300, 256)          296448    
_________________________________________________________________
dropout_1 (Dropout)          (None, 300, 256)          0         
_________________________________________________________________
bgru_2 (Bidirectional)       (None, 300, 256)          296448    
_________________________________________________________________
dropout_2 (Dropout)          (None, 300, 256)          0         
_________________________________________________________________
dense1 (TimeDistributed)     (None, 300, 1)            257 

In [ ]:
train_x = tx[:60000]
train_y = ty[:60000]
print(len(train_x),len(train_x[0]))

model.fit(train_x, train_y, epochs=10, batch_size=64,validation_split=0.2)
model.save('./BGRU/BGRU_model1.h5')

60000 300
Epoch 1/10
364/750 [=============>................] - ETA: 54:13 - loss: 0.5293 - accuracy: 0.7526 - precision: 0.7621 - recall: 0.8769 - true_positives: 12736.0000 - true_negatives: 4797.0000 - false_positives: 3975.0000 - false_negatives: 1788.0000

In [68]:
tt=int(np.random.randint(3000,8000))*10
ttx=tt+999
print(tt,ttx)
test_x = tx[30000+tt:30000+ttx]
test_y = ty[30000+tt:30000+ttx]
results = model.evaluate(test_x, test_y, batch_size=64)
print(results)


16/16 [==============================] - 17s 1s/step - loss: 0.6901 - accuracy: 0.6136 - precision: 0.5923 - recall: 0.9693 - true_positives: 536.0000 - true_negatives: 77.0000 - false_positives: 369.0000 - false_negatives: 17.0000
[0.6901264786720276, 0.6136136054992676, 0.5922651886940002, 0.9692586064338684, 536.0, 77.0, 369.0, 17.0]
35340 36339
16/16 [==============================] - 16s 1s/step - loss: 0.6866 - accuracy: 0.6807 - precision: 0.6807 - recall: 1.0000 - true_positives: 680.0000 - true_negatives: 0.0000e+00 - false_positives: 319.0000 - false_negatives: 0.0000e+00
[0.6866385340690613, 0.6806806921958923, 0.6806806921958923, 1.0, 680.0, 0.0, 319.0, 0.0]


In [37]:
x = tf.constant([[[ 4,  5,  6]],
                 [[10, 11, 12]]])
print(x.get_shape())
top_k = tf.nn.top_k(x, k=1, sorted=True, name=None)[0]
print(top_k)
shifted_output = tf.transpose(top_k, perm=[0, 2, 1])
print(shifted_output)
print(shifted_output.get_shape())

(2, 1, 3)
tf.Tensor(
[[[ 6]]

 [[12]]], shape=(2, 1, 1), dtype=int32)
tf.Tensor(
[[[ 6]]

 [[12]]], shape=(2, 1, 1), dtype=int32)
(2, 1, 1)


In [45]:
x = tf.constant([1., 2., 3., 4., 5.])
x = tf.reshape(x, [1, 1, 5])
print(x)
max_pool_1d = tf.keras.layers.MaxPooling1D(pool_size=2,
strides=2, padding='valid')
print(max_pool_1d)

tf.Tensor([[[1. 2. 3. 4. 5.]]], shape=(1, 1, 5), dtype=float32)
